In [71]:
# enable reload the module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
from mtr_inference import MTRInference
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import os
cfg_file = '/home/zixuzhang/Documents/Git/MTR/tools/cfgs/waymo/eval.yaml'
ckpt_path = '/home/zixuzhang/Documents/Git/MTR/model/checkpoint_epoch_30.pth'

# Interesting case:
1. Val, 6, 2d84b1ab55ab81d3, U-turn vs Left-turn
2. Val, 7, 3505972c220e6b3a, Turn vs Striaght for merging
3. Val, 25, 6d2a107f2e8390a, Merge to main road
4. Val, 26, ae92386e0686d4d6, Complicated 4-way stop
5. Val, 27, 3d00a93f0ba5e294, Both car merge to main road
6. Val, 36, 31d18fac4011375c, illegal merging to the main road

In [91]:

mtr_inference = MTRInference(cfg_file, ckpt_path)

index = 26

mtr_inference.generate_gif(index, 'gif')
